In [63]:
#!/usr/bin/env python
import math
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import pickle
from sklearn import datasets, metrics

In [64]:
def run_experiment(method, closure, X, y, X_test, y_test, init_step_size=1e-3, batch_size=10, max_epochs=50, num_restarts=10, verbose=False, seed=9513451):
    '''Run an experiment with multiple restarts and compute basic statistics from the runs.'''
    # set the experiment seed
    print("Running Experiment:")
    np.random.seed(seed)
    loss_results = []
    gradnorm_results = []

    n,d = X.shape
    n_test = X_test.shape[0]
    x_sum = np.zeros(d)

    # do the restarts
    for i in range(num_restarts):
        x, loss_record, gradnorm_record = method(closure, batch_size, X, y, init_step_size, n, d, max_epoch=max_epochs, verbose=verbose)
        x_sum += x
        loss_results.append(loss_record)
        gradnorm_results.append(gradnorm_record)

        if verbose:
            y_predict = np.sign(np.dot(X_test, x))
            print('Restart %d, Test accuracy: %f' % (i, (np.count_nonzero(y_test == y_predict)*1.0 / n_test)))

    # compute basic statistics from the runs
    x_mean = x_sum / num_restarts

    loss_results = np.stack(loss_results)
    loss_std = loss_results.std(axis=0)
    loss_mean = loss_results.mean(axis=0)

    gradnorm_results = np.stack(gradnorm_results)
    gradnorm_std = gradnorm_results.std(axis=0)
    gradnorm_mean = gradnorm_results.mean(axis=0)

    return x_mean, loss_mean, loss_std, gradnorm_mean, gradnorm_std

In [65]:
def make_closure(loss_fn, prior_prec=1e-2):
    '''Computes loss and gradient of the loss w.r.t. w
        Parameters:
            loss_fn: the loss function to use (logistic loss, hinge loss, squared error, etc)
            prior_prec: precision of the Gaussian prior (pass 0 to avoid regularization)
        Returns: a closure fn for computing the loss and gradient. '''

    def closure(w, X, y):
        '''Computes loss and gradient of the loss w.r.t. w
        Parameters:
            w: weight vector
            X: minibatch of input vectors
            y: labels for the input vectors
            prior_prec: precision of the Gaussian prior (pass 0 to avoid regularization)
        Returns: (loss, gradient)'''
        # change the Numpy Arrays into PyTorch Tensors
        X = torch.tensor(X)
        # Type of X is double, so y must be double.
        y = torch.tensor(y, dtype=torch.double)
        w = torch.tensor(w, requires_grad=True)

        # Compute the loss.
        loss = loss_fn(w, X, y) + (prior_prec / 2) * torch.sum(w**2)

        # compute the gradient of loss w.r.t. w.
        loss.backward()
        # Put the gradient and loss back into Numpy.
        grad = w.grad.detach().numpy()
        loss = loss.item()

        return loss, grad

    return closure

# PyTorch Loss Functions

def logistic_loss(w, X, y):
    ''' Logistic Loss'''
    n,d = X.shape
    return torch.mean(torch.log(1 + torch.exp(-torch.mul(y, torch.matmul(X, w)))))

def squared_hinge_loss(w, X, y):
    n,d = X.shape
    '''Squared Hinge Loss '''
    return torch.mean((torch.max( torch.zeros(n,dtype=torch.double) , torch.ones(n,dtype=torch.double) - torch.mul(y, torch.matmul(X, w))))**2 )

def squared_loss(w, X, y):
    n,d = X.shape
    '''Squared Loss'''
    return torch.mean(( y - torch.matmul(X, w) )**2)

### predefined loss closures ###

logistic_closure_no_l2 = make_closure(logistic_loss, 0)
logistic_closure_default_l2 = make_closure(logistic_loss)

In [66]:
def data_load(data_dir, dataset_num, n = 0, d = 0, is_subsample = 0, is_kernelize = 0, test_prop=0.2, split_seed=9513451):

    if dataset_num == 0:
        data_name = 'quantum'

    elif dataset_num == 1:
        data_name = 'rcv1'

    elif dataset_num == 2:
        data_name = 'protein'

    elif dataset_num == 3:
        data_name = 'news'

    elif dataset_num == 4:
        data_name = 'mushrooms'

    elif dataset_num == 5:
        data_name = 'splice'

    elif dataset_num == 6:
        data_name = 'ijcnn'

    elif dataset_num == 7:
        data_name = 'w8a'

    elif dataset_num == 8:
        data_name = 'covtype'

    elif dataset_num == -1:
        data_name = 'synthetic'

    if (dataset_num >= 0):

        # real data
        data = pickle.load(open(data_dir + data_name +'.pkl', 'rb'), encoding = "latin1")

        # load real dataset
        A = data[0].toarray()

        if dataset_num < 4:
            y = data[1].toarray().ravel()
        else:
            y = data[1]

    else:

        A, y, w_true = create_dataset(n,d,0.5)

        # generate synthetic data - according to the BB paper
#         x = np.random.randn(n, d)
#         w_true = np.random.randn(d)
#         y = np.sign(np.dot(x, w_true))

    # subsample
    if is_subsample == 1:
        A = A[:n,:]
        y = y[:n]

    # split dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=test_prop, random_state=split_seed)

    if is_kernelize == 1:
        # Form kernel
        A_train, A_test = kernelize(X_train, X_test, dataset_num, data_dir=data_dir)
    else:
        A_train = X_train
        A_test = X_test

    print('Loaded ', data_name ,' dataset.')

    return A_train, y_train, A_test, y_test

def kernelize(X, X_test, dataset_num, kernel_type=0, data_dir="./Data"):

    n = X.shape[0]

    fname = data_dir + '/Kernel_' + str(n) + '_' + str(dataset_num) + '.p'

    if os.path.isfile(fname):

        print('Reading file ', fname)
        X_kernel, X_test_kernel = pickle.load( open( fname, "rb" ) )

    else:
        if kernel_type == 0:
            X_kernel = RBF_kernel(X, X)
            X_test_kernel = RBF_kernel(X_test, X)
            print('Formed the kernel matrix')

        pickle.dump( (X_kernel, X_test_kernel) , open( fname, "wb" ) )

    return X_kernel, X_test_kernel

def RBF_kernel( A, B, sigma = 1.0 ):

    distance_2 = np.square(  metrics.pairwise.pairwise_distances( X = A, Y = B, metric='euclidean'  )   )
    K = np.exp( -1 * np.divide( distance_2, (2 * (sigma**2)) )  )

    return K


def create_dataset(n,d,gamma):
# create synthetic dataset using the python utility
# X, y = datasets.make_classification(n_samples=n, n_features=d,n_informative = d, n_redundant = 0, class_sep = 2.0 )
# convert into -1/+1
# y = 2 * y - 1

# create linearly separable dataset with margin gamma
#w_star = np.random.random((d,1))
    w_star = np.random.normal(0,1,(d,1))
# normalize w_star
    w_star = w_star / np.linalg.norm(w_star)

    num_positive = 0
    num_negative = 0
    count = 0

    X = np.zeros((n,d))
    y = np.zeros((n))

    while(1):

        x = np.random.normal( 1,1,(1,d) )
        # normalize x s.t. || x ||_2 = 1
        x = x / np.linalg.norm(x)

        temp = np.dot( x, w_star )
        margin = abs( temp )
        sig = np.sign( temp )

        if margin > gamma * np.linalg.norm(w_star):

            if count % 2 == 0:

                # generate positive
                if sig > 0:
                    X[count, :] = x
                else:
                    X[count, :] = -x
                y[ count ] = + 1

            else:

                # generate negative
                if sig < 0:
                    X[count, :] = x
                else:
                    X[count,:] = -x
                y[ count ] = - 1

            count = count + 1

        if count == n:
            break

    return X, y, w_star


In [67]:
def make_minibatches(n, m, minibatch_size):
    ''' Create m minibatches from the training set by sampling without replacement.
        This function may sample the training set multiple times.
    Parameters:
        n: the number of examples in the dataset
        m: number of minibatches to generate
        batch_size: size of the desired minibatches'''

    k = math.ceil(m * minibatch_size / n)
    batches = []
    for i in range(k):
        batches += minibatch_data(n, minibatch_size)

    return batches


def minibatch_data(n, batch_size):
    '''Splits training set into minibatches by sampling **without** replacement.
    This isn't performant for large datasets (e.g. we should switch to PyTorch's streaming data loader eventually).
    Parameters:
        n: the number of examples in the dataset
        batch_size: size of the desired minibatches'''
    # shuffle training set indices before forming minibatches
    indices = np.arange(n)
    np.random.shuffle(indices)

    batches = []
    num_batches = math.floor(n / batch_size)
    # split the training set into minibatches
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        stop_index = (batch_num + 1) * batch_size

        # create a minibatch
        batches.append(indices[start_index:stop_index])

    # generate a final, smaller batch if the batch_size doesn't divide evenly into n
    if num_batches != math.ceil(n / batch_size):
        batches.append(indices[stop_index:])

    return batches

def reset(model):
    # reset the model
    for param in model.parameters():
        param.data = torch.zeros_like(param)
    loss_results = []
    gradnorm_results = []

    return model

In [68]:
def plotting(results, max_epochs):
    plt.figure()

    offset = 0
    colors = ['r', 'b', 'g','k','cyan']
    labels = ['SVRG-BB', 'SVRG' ]
    x = range(max_epochs)

    for i in range(len(labels)):
        plt.plot( x, ( results[i,:] ), color = colors[i], label = labels[i] )

    plt.xlabel('Number of Effective Passes')
    plt.ylabel('Loss')
    plt.legend(loc='best')
    plt.show()

def plot_shaded_error_bars(results_mean, results_std, max_epochs, label="Loss", colors=None, labels=None):
    fig = plt.figure()
    offset = 0
    if colors is None:
        colors = ['r', 'b', 'g','k','cyan']
    if labels is None:
        labels = ['SVRG-BB', 'SVRG']

    x = range(max_epochs)

    for i in range(results_mean.shape[0]):
        plt.plot( x, ( results_mean[i,:] ), color = colors[i], label = labels[i] )
        plt.fill_between(x, (results_mean[i,:] - results_std[i,:]), (results_mean[i,:] + results_std[i,:]), color=colors[i], alpha=0.5)

    plt.xlabel('Number of Effective Passes')
    plt.ylabel(label)
    plt.legend(loc='best')

    return fig

In [69]:
def svrg_bb(closure, batch_size, D, labels, init_step_size, n, d, max_epoch=100, m=0, x0=None, verbose=True):
    """
        SVRG with Barzilai-Borwein step size for solving finite-sum problems
        Closure: a PyTorch-style closure returning the objective value and it's gradient.
        batch_size: the size of minibatches to use.
        D: the set of input vectors (usually X).
        labels: the labels corresponding to the inputs D.
        init_step_size: initial step size
        n, d: size of the problem
    """
    if not isinstance(m, int) or m <= 0:
        m = n
        if verbose:
            print('Info: set m=n by default')

    if x0 is None:
        x = np.zeros(d)
    elif isinstance(x0, np.ndarray) and x0.shape == (d, ):
        x = x0.copy()
    else:
        raise ValueError('x0 must be a numpy array of size (d, )')

    step_size = init_step_size

    LOSS = np.zeros((max_epoch))
    GRAD_NORM = np.zeros((max_epoch))

    for k in range(max_epoch):
        loss, full_grad = closure(x, D, labels)
        x_tilde = x.copy()
        # estimate step size by BB method
        if k > 0:
            s = x_tilde - last_x_tilde
            y = full_grad - last_full_grad
            step_size = np.linalg.norm(s)**2 / np.dot(s, y) / m

        last_full_grad = full_grad
        last_x_tilde = x_tilde

        if verbose:
            output = 'Epoch.: %d, Step size: %.2e, Grad. norm: %.2e' % \
                     (k, step_size, np.linalg.norm(full_grad))
            output += ', Func. value: %e' % loss
            print(output)

        # Add termination condition based on the norm of full gradient
        # Without this, "np.dot(s, y)" can underflow and produce divide-by-zero errors.
        if np.linalg.norm(full_grad) <= 1e-14:
            return x, LOSS, GRAD_NORM

        LOSS[k] = loss
        GRAD_NORM[k] = np.linalg.norm(full_grad)

        # Create Minibatches:
        minibatches = make_minibatches(n, m, batch_size)
        for i in range(m):
            # get the minibatch for this iteration
            indices = minibatches[i]
            Di, labels_i = D[indices, :], labels[indices]

            # compute the gradients:
            x_grad = closure(x, Di, labels_i)[1]
            x_tilde_grad = closure(x_tilde, Di, labels_i)[1]

            x -= step_size * (x_grad - x_tilde_grad + full_grad)

    return x, LOSS, GRAD_NORM


In [70]:
def svrg(closure, batch_size, D, labels, init_step_size, n, d, max_epoch=100, m=0, x0=None, verbose=True):
    """
        SVRG with fixed step size for solving finite-sum problems
        Closure: a PyTorch-style closure returning the objective value and it's gradient.
        batch_size: the size of minibatches to use.
        D: the set of input vectors (usually X).
        labels: the labels corresponding to the inputs D.
        init_step_size: step-size to use
        n, d: size of the problem
    """
    step_size = init_step_size
    if not isinstance(m, int) or m <= 0:
        m = n
        if verbose:
            print('Info: set m=n by default')

    if x0 is None:
        x = np.zeros(d)
    elif isinstance(x0, np.ndarray) and x0.shape == (d, ):
        x = x0.copy()
    else:
        raise ValueError('x0 must be a numpy array of size (d, )')

    LOSS = np.zeros((max_epoch))
    GRAD_NORM = np.zeros((max_epoch))

    for k in range(max_epoch):
        loss, full_grad = closure(x, D, labels)
        x_tilde = x.copy()

        last_full_grad = full_grad
        last_x_tilde = x_tilde

        if verbose:
            output = 'Epoch.: %d, Grad. norm: %.2e' % \
                     (k, np.linalg.norm(full_grad))
            output += ', Func. value: %e' % loss
            print(output)

        if np.linalg.norm(full_grad) <= 1e-14:
            return x, LOSS, GRAD_NORM

        LOSS[k] = loss
        GRAD_NORM[k] = np.linalg.norm(full_grad)

        # Create Minibatches:
        minibatches = make_minibatches(n, m, batch_size)
        for i in range(m):
            # get the minibatch for this iteration
            indices = minibatches[i]
            Di, labels_i = D[indices, :], labels[indices]

            # compute the gradients:
            x_grad = closure(x, Di, labels_i)[1]
            x_tilde_grad = closure(x_tilde, Di, labels_i)[1]

            x -= step_size * (x_grad - x_tilde_grad + full_grad)

    return x, LOSS, GRAD_NORM


In [71]:
# dataset options
dataset_num = -1
data_dir = './Data/'

is_subsample = 0
is_kernelize = 0
subsampled_n = -1

# set the seed for reproducibility
np.random.seed(6162647)

In [72]:
# optimization options
max_epochs = 20
num_restarts = 1
num_variants = 2
batch_size = 100
verbose = True

In [73]:
# problem size when generating synthetic data
if dataset_num == -1:
    n, d = 1000, 20
    print(is_kernelize)
    A, y, A_test, y_test = data_load(data_dir, dataset_num,n, d)
else:
    if is_subsample == 1:
        n = subsampled_n
    else:
        n = 0
    if is_kernelize == 1:
        d = n
    else:
        d = 0
        
    A, y, A_test, y_test = data_load(data_dir, dataset_num, n,d, is_subsample, is_kernelize)

0
Loaded  synthetic  dataset.


In [74]:
results_mean = np.zeros((num_variants, max_epochs))
results_std = np.zeros((num_variants, max_epochs))

results2_mean = np.zeros((num_variants, max_epochs))
results2_std = np.zeros((num_variants, max_epochs))

In [75]:
# # test SVRG-BB
# x0 = np.random.rand(d)
print('Begin to run SVRG-BB:')
init_eta = 1e-3
x_mean, results_mean[0, :], results_std[0, :], results2_mean[0, :], results2_std[0, :] = run_experiment(svrg_bb, logistic_closure_no_l2, A, y, A_test, y_test, init_eta, batch_size, max_epochs, num_restarts, verbose=verbose)

Begin to run SVRG-BB:
Running Experiment:
Info: set m=n by default
Epoch.: 0, Step size: 1.00e-03, Grad. norm: 3.18e-01, Func. value: 6.931472e-01
Epoch.: 1, Step size: 1.09e-02, Grad. norm: 2.90e-01, Func. value: 6.194315e-01
Epoch.: 2, Step size: 1.35e-02, Grad. norm: 1.34e-01, Func. value: 2.735625e-01
Epoch.: 3, Step size: 2.50e-02, Grad. norm: 7.85e-02, Func. value: 1.601749e-01
Epoch.: 4, Step size: 4.37e-02, Grad. norm: 4.49e-02, Func. value: 8.963916e-02
Epoch.: 5, Step size: 7.49e-02, Grad. norm: 2.54e-02, Func. value: 4.955512e-02
Epoch.: 6, Step size: 1.28e-01, Grad. norm: 1.44e-02, Func. value: 2.760178e-02
Epoch.: 7, Step size: 2.21e-01, Grad. norm: 8.16e-03, Func. value: 1.556533e-02
Epoch.: 8, Step size: 3.83e-01, Grad. norm: 4.66e-03, Func. value: 8.842998e-03
Epoch.: 9, Step size: 6.67e-01, Grad. norm: 2.66e-03, Func. value: 5.045063e-03
Epoch.: 10, Step size: 1.16e+00, Grad. norm: 1.52e-03, Func. value: 2.884843e-03
Epoch.: 11, Step size: 2.03e+00, Grad. norm: 8.70e-0

In [ ]:
# # test SVRG-BB
# x0 = np.random.rand(d)
print('Begin to run SVRG:')
init_eta = 1e-3
x_mean, results_mean[1, :], results_std[1, :], results2_mean[1, :], results2_std[1, :] = run_experiment(svrg, logistic_closure_no_l2, A, y, A_test, y_test, init_eta, batch_size, max_epochs, num_restarts, verbose=verbose)

Begin to run SVRG:
Running Experiment:
Info: set m=n by default
Epoch.: 0, Grad. norm: 3.18e-01, Func. value: 6.931472e-01
Epoch.: 1, Grad. norm: 2.90e-01, Func. value: 6.194315e-01
Epoch.: 2, Grad. norm: 2.65e-01, Func. value: 5.579821e-01
Epoch.: 3, Grad. norm: 2.43e-01, Func. value: 5.064780e-01
Epoch.: 4, Grad. norm: 2.24e-01, Func. value: 4.630032e-01
Epoch.: 5, Grad. norm: 2.07e-01, Func. value: 4.260135e-01
Epoch.: 6, Grad. norm: 1.92e-01, Func. value: 3.942800e-01
Epoch.: 7, Grad. norm: 1.79e-01, Func. value: 3.668303e-01
Epoch.: 8, Grad. norm: 1.67e-01, Func. value: 3.428959e-01
Epoch.: 9, Grad. norm: 1.57e-01, Func. value: 3.218677e-01


In [ ]:
fig = plot_shaded_error_bars(results_mean, results_std, max_epochs)
plt.show()

In [ ]:
fig = plot_shaded_error_bars(results2_mean, results2_std, max_epochs)
plt.show()